# CLIP - Dataset

## 0. imports

In [1]:
%load_ext jupyter_black

In [2]:
import os
import random

import cv2
import numpy as np
import pandas as pd

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModel

from sklearn.model_selection import train_test_split

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## 1. Dataset - Line by Line

### 1.1 image - transform

In [4]:
def get_transform(img_size: int, max_pixel_value: float = 255.0, stage: str = "train"):
    if stage == "train":
        transform = A.Compose(
            [
                A.Resize(img_size, img_size, always_apply=True),
                A.Normalize(max_pixel_value=max_pixel_value, always_apply=True),
                ToTensorV2(),
            ]
        )
    else:
        transform = A.Compose(
            [
                A.Resize(img_size, img_size, always_apply=True),
                A.Normalize(max_pixel_value=max_pixel_value, always_apply=True),
                ToTensorV2(),
            ]
        )

    return transform

In [5]:
transform = get_transform(img_size=224)

### 1.2 text - tokenizer

In [6]:
tokenizer_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

### 1.3 data load

In [19]:
img_dir = "../data/images"
caption_path = "../data/captions.csv"
info_df = pd.read_csv(caption_path, encoding_errors="ignore")

In [27]:
info_df["image_path"] = info_df["image_name"].apply(
    lambda row: f"{row.split('_')[0]}/{row}"
)

In [29]:
info_df.head()

,image_name,caption,label_game,label_genre,image_path
0,bingsu_img_0.png,The image shows a lobby from the popular multi...,among_us,strategy,bingsu/bingsu_img_0.png
1,bingsu_img_0.png,"The lobby in ""Among Us"" is depicted in the ima...",among_us,strategy,bingsu/bingsu_img_0.png
2,bingsu_img_0.png,The lobby in the image is from the well-known ...,among_us,strategy,bingsu/bingsu_img_0.png
3,bingsu_img_0.png,The picture displays a waiting area in the wel...,among_us,strategy,bingsu/bingsu_img_0.png
4,bingsu_img_0.png,The picture displays a lobby in the well-liked...,among_us,strategy,bingsu/bingsu_img_0.png


### 1.4 img & text -  preprocess

In [30]:
idx = random.choice(range(len(info_df)))

image_name = info_df.iloc[idx]["image_name"]
caption = info_df.iloc[idx]["caption"]
game_name = info_df.iloc[idx]["label_game"]
game_genre = info_df.iloc[idx]["label_genre"]
image_path = info_df.iloc[idx]["image_path"]

In [24]:
max_length = 200

captions = info_df["caption"].tolist()
encoded_captions = tokenizer(
    captions, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
)

In [25]:
encoded_captions["input_ids"].shape

torch.Size([43392, 145])

In [31]:
img_path = os.path.join(img_dir, image_path)

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img = transform(image=img)["image"]

In [34]:
item = {k: v[idx] for k, v in encoded_captions.items()}
item["image"] = img
item["caption"] = caption

## 2. Dataset - Class

In [40]:
class CLIPDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        img_dir: str,
        tokenizer,
        transform,
        txt_max_length: int = 200,
    ):
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.transform = transform
        self.txt_max_length = txt_max_length

        # dataframe
        self.data = df
        self.data["image_path"] = self.data["image_name"].apply(
            lambda row: f"{row.split('_')[0]}/{row}"
        )

        # encoded_captions
        captions = self.data["caption"].tolist()
        self.encoded_captions = self.tokenizer(
            captions,
            padding=True,
            truncation=True,
            max_length=self.txt_max_length,
            return_tensors="pt",
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name = self.data.iloc[idx]["image_name"]
        caption = self.data.iloc[idx]["caption"]
        game_name = self.data.iloc[idx]["label_game"]
        game_genre = self.data.iloc[idx]["label_genre"]
        image_path = self.data.iloc[idx]["image_path"]

        # txt prep
        item = {k: v[idx] for k, v in self.encoded_captions.items()}

        # img prep
        img_path = os.path.join(self.img_dir, image_path)

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)["image"]

        item["image"] = img
        item["caption"] = caption
        item["game_name"] = game_name
        item["game_genre"] = game_genre

        return item

In [41]:
data_path = "../data/captions.csv"
img_dir = "../data/images/"
tokenizer_name = "bert-base-uncased"
img_size = 224
txt_max_length = 200


tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
transform = get_transform(img_size=img_size)

In [42]:
test_size = 0.2
val_size = 0.2

df = pd.read_csv(data_path, encoding_errors="ignore")

train_df, test_df = train_test_split(df, test_size=test_size, shuffle=True)
train_df, val_df = train_test_split(train_df, test_size=test_size, shuffle=True)

In [43]:
dataset = CLIPDataset(train_df, img_dir, tokenizer, transform, txt_max_length)

In [44]:
batch_size = 32

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [45]:
batch = next(iter(dataloader))

In [46]:
batch

{'input_ids': tensor([[ 101, 1999, 2023,  ...,    0,    0,    0],
         [ 101, 1999, 1996,  ...,    0,    0,    0],
         [ 101, 1996, 3746,  ...,    0,    0,    0],
         ...,
         [ 101, 1999, 1996,  ...,    0,    0,    0],
         [ 101, 1037, 2447,  ...,    0,    0,    0],
         [ 101, 1999, 1996,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'image': tensor([[[[-0.5767, -0.5767, -0.5767,  ..., -1.3644, -1.3644, -1.3644],
           [-0.5596, -0.5596, -0.5596,  ..., -1.3644, -1.3644, -1.3644],
          

## 3. DataModule

In [47]:
class CLIPDataModule:
    def __init__(
        self,
        data_path: str,
        img_dir: str,
        tokenizer_name: str,
        img_size: int = 224,
        txt_max_length: int = 200,
        val_size: float = 0.2,
        test_size: float = 0.2,
        batch_size: int = 32,
        num_workers: int = 4,
    ):
        self.data_path = data_path
        self.img_dir = img_dir
        self.tokenizer_name = tokenizer_name
        self.img_size = img_size
        self.txt_max_length = txt_max_length
        self.val_size = val_size
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.setup()

    def setup(self):
        # load data
        self.df = pd.read_csv(self.data_path, encoding_errors="ignore")
        self.df["image_path"] = self.df["image_name"].apply(
            lambda row: f"{row.split('_')[0]}/{row}"
        )

        # tokenizer & transform
        self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)
        self.train_transform = get_transform(img_size=self.img_size, stage="train")
        self.test_transform = get_transform(img_size=self.img_size, stage="test")

        # train/val/test split
        train_df, test_df = train_test_split(
            self.df, test_size=self.test_size, shuffle=True
        )
        train_df, val_df = train_test_split(
            train_df, test_size=self.test_size, shuffle=True
        )

        # train/val/test set
        self.trainset = CLIPDataset(
            train_df,
            self.img_dir,
            self.tokenizer,
            self.train_transform,
            self.txt_max_length,
        )
        self.valset = CLIPDataset(
            val_df,
            self.img_dir,
            self.tokenizer,
            self.test_transform,
            self.txt_max_length,
        )
        self.testset = CLIPDataset(
            test_df,
            self.img_dir,
            self.tokenizer,
            self.test_transform,
            self.txt_max_length,
        )

    def train_dataloader(self):
        return DataLoader(
            self.trainset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.testset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )

In [48]:
dm_params = {
    "data_path": "../data/captions.csv",
    "img_dir": "../data/images/",
    "tokenizer_name": "bert-base-uncased",
    "img_size": 224,
    "txt_max_length": 200,
    "val_size": 0.2,
    "test_size": 0.2,
    "batch_size": 32,
    "num_workers": 4,
}

dm = CLIPDataModule(**dm_params)

In [49]:
train_batch = next(iter(dm.train_dataloader()))
val_batch = next(iter(dm.val_dataloader()))
test_batch = next(iter(dm.test_dataloader()))

In [50]:
train_batch.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'image', 'caption', 'game_name', 'game_genre'])